<a href="https://colab.research.google.com/github/VicenteLavagnino/github-hack-feb25/blob/main/Copy_of_%5Bexternal%5D_Healthcare_Generation_w_Reasoning_Mode_DeepSeek_(v3_%26_R1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1njzOgHWgguSuA732RYROJDiCua3hGO_R?usp=sharing)

# Introduction to Structured Responses for Healthcare Data

This script demonstrates how to use **structured responses** with DeepSeek Models (V3 and R1) on Fireworks AI to generate reliable and structured healthcare data.


### Why Use Structured Responses?
- **Predictability**: Enforces medical data constraints to ensure logical outputs.
- **Automation**: Structured outputs can be used for patient record generation, automated reports, and clinical decision support.
- **Integration**: JSON and Grammar Mode allow seamless integration into electronic health record (EHR) systems and research databases.

### What You'll Learn
1. **Basic API Setup**: Connecting to Fireworks AI and using DeepSeek v3 and R1.
2. **Grammar Mode**: Generating structured healthcare data within predefined constraints.
3. **JSON Mode**: Ensuring machine-readable structured responses.
4. **Reasoning JSON Mode**: Capturing the model's thought process alongside structured responses.

### Setup

Install all the dependencies and import the required python modules.

In [2]:
!pip3 install --upgrade fireworks-ai pydantic python-dotenv > /dev/null

### Setup your API Key

In order to use the DeepSeek models, you must first obtain Fireworks API Keys. If you don't already have one, create a [Fireworks AI](https://fireworks.ai) account. Under Account Settings, click on [API Keys](https://fireworks.ai/api-keys) to create one.

In [10]:
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv  # Import dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Fireworks client using API key from .env
client = Fireworks(api_key=FIREWORKS_API_KEY)

# 0️⃣ **Basic API Call to DeepSeek v3**

### Chat Completions API
This is a simple test call to ensure the Fireworks API is working correctly.

In [32]:
model_name = "accounts/fireworks/models/deepseek-v3"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "What are the latest advancements in AI for healthcare?"},
    ],
)
print(response.choices[0].message.content)

The field of AI in healthcare has seen rapid advancements in recent years, with innovations transforming diagnostics, treatment, patient care, and drug discovery. Here are some of the latest advancements as of 2023:

### 1. **AI in Medical Imaging**
   - **Enhanced Image Analysis**: AI algorithms are now more accurate in detecting anomalies in medical images such as X-rays, MRIs, and CT scans. For example, advancements in deep learning have improved the detection of cancers, cardiovascular diseases, and neurological disorders.
   - **Explainable AI (XAI)**: Efforts are being made to make AI-driven imaging tools more interpretable, allowing clinicians to understand how the AI arrives at its conclusions.

### 2. **Drug Discovery and Development**
   - **Generative AI for Molecule Design**: AI models like AlphaFold and others are being used to predict protein structures and design new drug candidates more efficiently.
   - **Virtual Screening**: AI is speeding up the process of identifyin

# 1️⃣ **Grammar Mode with BNF for Healthcare Data**

### **What is Grammar Mode?**
Grammar Mode ensures that the model outputs responses that conform to a strict structure, preventing unexpected variations in responses. This is particularly useful for generating structured data, such as medical records.

#### **How Grammar Mode Works**
- Uses a **GBNF (Grammar-Based Normal Form)** to constrain the model's output.
- Ensures the output adheres to predefined rules for valid healthcare data.

Read more about [GBNF](https://github.com/ggerganov/llama.cpp/tree/master/grammars), and how to [use it](https://docs.fireworks.ai/structured-responses/structured-output-grammar-based).


In [37]:
# Define the grammar to enforce structured output for triage ranking
triage_grammar = """
root ::= patient-record

patient-record ::= "Patient ID: " patient_id " Age: " age " Gender: " gender " Symptoms: " symptoms " Vitals: " vitals

patient_id ::= ("P001" | "P002" | "P003" | "P004" | "P005")
age ::= ("18" | "25" | "40" | "60" | "75")
gender ::= ("Male" | "Female" | "Non-binary")
symptoms ::= ("Chest pain" | "Shortness of breath" | "High fever" | "Unconscious" | "Severe bleeding")
vitals ::= ("Normal BP, 98% Oxygen" | "High BP, Low Oxygen" | "Rapid Heart Rate, High Fever" | "Unstable BP, Loss of Consciousness")
"""

In [41]:
# Call the model to enforce structured triage ranking
response_grammar = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": (
                "You are an AI-powered emergency triage system for healthcare. \n"
                "Given a patient's structured data (age, symptoms, vitals), assess their emergency priority. \n"
                "Analyze the provided information and return a structured JSON output with:\n\n"
                "- 'patient_id': unique patient identifier\n"
                "- 'mortality_risk': probability of death without immediate care (0-100%)\n"
                "- 'priority_level': urgency level ('Low', 'Medium', 'High', 'Critical')\n"
                "- 'recommended_action': next steps ('Monitor', 'Immediate Attention', 'ICU Admission', 'Surgery Required')\n"
                "- 'justification': reasoning for the decision\n\n"
                "Format the output strictly in JSON like this example:\n"
                "{\n"
                '  "patient_id": "P001",\n'
                '  "mortality_risk": 85,\n'
                '  "priority_level": "Critical",\n'
                '  "recommended_action": "ICU Admission",\n'
                '  "justification": "Patient is unconscious with unstable vitals."\n'
                "}\n"
                "Do NOT include any additional text before or after the JSON response."
            ),
        }
    ],
    max_tokens=500,
)

In [42]:
import json

# Extract AI-generated triage details
grammar_output = response_grammar.choices[0].message.content

print("🚑 AI-Generated Triage Output:", grammar_output)

# Parse JSON safely
try:
    triage_data = json.loads(grammar_output)
    print(f"🔴 Patient ID: {triage_data['patient_id']}")
    print(f"⚠️ Mortality Risk: {triage_data['mortality_risk']}%")
    print(f"🏥 Priority Level: {triage_data['priority_level']}")
    print(f"🩺 Recommended Action: {triage_data['recommended_action']}")
    print(f"📌 Justification: {triage_data['justification']}")
except json.JSONDecodeError:
    print("⚠️ AI response is not in JSON format, please check output manually.")

🚑 AI-Generated Triage Output: {
  "patient_id": "P002",
  "mortality_risk": 30,
  "priority_level": "Medium",
  "recommended_action": "Immediate Attention",
  "justification": "Patient exhibits moderate symptoms with slightly elevated vitals, indicating potential risk if left untreated."
}
🔴 Patient ID: P002
⚠️ Mortality Risk: 30%
🏥 Priority Level: Medium
🩺 Recommended Action: Immediate Attention
📌 Justification: Patient exhibits moderate symptoms with slightly elevated vitals, indicating potential risk if left untreated.


# 2️⃣ **JSON Mode for Healthcare Data**

JSON mode is a subset of grammar mode and is a feature that allows you to specify a JSON schema to force a language model to respond in a structured JSON format. This has several benefits:

* Clarity and precision in responses
* Easy parsing and processing of output
* Improved integration with healthcare systems
* Standardized format for consistent data handling

In [15]:
from pydantic import BaseModel, Field
from typing import List

In [43]:
# Define the structure for triage response
class TriageRecord(BaseModel):
    patient_id: str
    age: int
    gender: str
    symptoms: str
    vitals: str
    mortality_risk: int  # 0-100 scale
    priority_level: str  # "Low", "Medium", "High", "Critical"
    recommended_action: str  # "Monitor", "Immediate Attention", "ICU Admission", "Surgery Required"
    justification: str  # Explanation of triage decision

In [44]:
# API Call - JSON Mode for Triage Ranking
response_json = client.chat.completions.create(
    model=model_name,
    response_format={"type": "json_object", "schema": TriageRecord.model_json_schema()},
    messages=[
        {
            "role": "user",
            "content": (
                "You are an AI-powered triage system. Based on the given structured patient data, "
                "analyze their condition and provide a triage assessment.\n\n"
                "Ensure the output strictly follows the JSON format defined by the schema."
            ),
        }
    ],
    max_tokens=500,
)

# Print AI-generated triage response
print(response_json.choices[0].message.content)

{
"patient_id": "PAT123456",
"age": 34,
"gender": "Female",
"symptoms": "chest pain, shortness of breath, dizziness",
"vitals": "BP: 140/90, HR: 110, Temp: 98.6°F, O2: 92%",
"mortality_risk": 10,
"priority_level": "High",
"recommended_action": "Immediate medical attention required. Admit to emergency department for further evaluation and management."
,
"justification": "Patient presents with symptoms of chest pain, shortness of breath, and dizziness, which could indicate a cardiac event given the elevated heart rate and low oxygen saturation. The high priority level is due to the potential for life-threatening conditions such as myocardial infarction or pulmonary embolism."
}


# 3️⃣ **Reasoning Model JSON Mode for Healthcare Data**

### 🧠 **Reasoning Model JSON Mode for Healthcare Data**  

#### 🔹 **What is Reasoning JSON Mode?**  
Reasoning JSON Mode extends structured JSON generation by adding an **explanation of the model’s decision-making process**. This is done by generating a **reasoning section enclosed within `<think>...</think>` tags**, followed by a structured JSON response.

This approach enhances transparency and interpretability by providing insight into **why** the model made certain clinical recommendations.

#### 🔹 **Why Use Reasoning JSON Mode for Healthcare Data?**  
For tasks like **clinical documentation, medical decision support, or patient record validation**, Reasoning JSON Mode provides:

* ✅ **Explainability** → Understand why certain diagnoses and treatments were selected.  
* ✅ **Debugging Capability** → Identify potential inconsistencies in AI-generated medical recommendations.  
* ✅ **Structured Data** → JSON output is ready for integration into healthcare databases.


In [18]:
import json
import re
from pydantic import BaseModel
from openai import OpenAI

In [45]:
# Prepare the user input for triage evaluation
user_input = (
    "You are an AI-powered emergency triage system. "
    "Given a patient's structured data (age, symptoms, vitals), assess their emergency priority. "
    "Analyze the provided information and provide:\n\n"
    "<think> A short explanation of the triage decision, enclosed in <think>...</think> tags. </think>\n\n"
    "Then return structured JSON in the format of the TriageRecord schema."
)

# Construct the messages payload
messages = [{"role": "user", "content": user_input}]

# Make the API call to the model
response = client.chat.completions.create(
    model="accounts/fireworks/models/deepseek-r1",
    messages=messages,
    response_format={"type": "json_object", "schema": TriageRecord.model_json_schema()},
    max_tokens=3000,
)

# Extract the content of the response
response_content = response.choices[0].message.content
print("📄 AI Response Content:\n", response_content)

📄 AI Response Content:
 <think>
Okay, let's tackle this. The patient is 67 years old. Their symptoms include chest pain, shortness of breath, and dizziness. Vitals: blood pressure is 160/95, heart rate 110 bpm, oxygen saturation 92%. Age is a factor here, as older adults are at higher risk for cardiovascular issues. Chest pain coupled with shortness of breath and elevated heart rate raises concern for possible cardiac event like a heart attack. The high blood pressure and low oxygen saturation (92% is below normal, which is usually 95%+) add to the urgency. Dizziness might indicate inadequate blood flow or oxygenation. Tachycardia (HR 110) and hypertension could be stress responses, but combined with the symptoms, it's a red flag. Immediate evaluation is needed to rule out life-threatening conditions like myocardial infarction or pulmonary embolism. Prioritize as emergent.
</think>
{
"patient_id": "12345",
"age": 67,
"gender": "male",
"symptoms": "chest pain, shortness of breath, dizzi

In [46]:
# Extract the reasoning part enclosed in <think>...</think> tags
reasoning_match = re.search(r"<think>(.*?)</think>", response_content, re.DOTALL)
reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."

print("🧠 AI Reasoning:\n", reasoning)

🧠 AI Reasoning:
 Okay, let's tackle this. The patient is 67 years old. Their symptoms include chest pain, shortness of breath, and dizziness. Vitals: blood pressure is 160/95, heart rate 110 bpm, oxygen saturation 92%. Age is a factor here, as older adults are at higher risk for cardiovascular issues. Chest pain coupled with shortness of breath and elevated heart rate raises concern for possible cardiac event like a heart attack. The high blood pressure and low oxygen saturation (92% is below normal, which is usually 95%+) add to the urgency. Dizziness might indicate inadequate blood flow or oxygenation. Tachycardia (HR 110) and hypertension could be stress responses, but combined with the symptoms, it's a red flag. Immediate evaluation is needed to rule out life-threatening conditions like myocardial infarction or pulmonary embolism. Prioritize as emergent.


In [47]:
# Extract the JSON part that follows after the reasoning
json_match = re.search(r"</think>\s*(\{.*\})", response_content, re.DOTALL)
json_str = json_match.group(1).strip() if json_match else "{}"

# Parse the JSON string into a Pydantic model
try:
    triage_record = TriageRecord.model_validate_json(json_str)
except Exception as e:
    print("⚠️ JSON Parsing Error:", e)
    triage_record = None

In [48]:
# Output the reasoning and the parsed JSON data
print("🩺 Reasoning for Triage Decision:")
print(reasoning)

if triage_record:
    print("\n🚑 Triage Record (Structured JSON Data):")
    print(triage_record.model_dump_json(indent=4))
else:
    print("\n⚠️ No valid triage data extracted.")

🩺 Reasoning for Triage Decision:
Okay, let's tackle this. The patient is 67 years old. Their symptoms include chest pain, shortness of breath, and dizziness. Vitals: blood pressure is 160/95, heart rate 110 bpm, oxygen saturation 92%. Age is a factor here, as older adults are at higher risk for cardiovascular issues. Chest pain coupled with shortness of breath and elevated heart rate raises concern for possible cardiac event like a heart attack. The high blood pressure and low oxygen saturation (92% is below normal, which is usually 95%+) add to the urgency. Dizziness might indicate inadequate blood flow or oxygenation. Tachycardia (HR 110) and hypertension could be stress responses, but combined with the symptoms, it's a red flag. Immediate evaluation is needed to rule out life-threatening conditions like myocardial infarction or pulmonary embolism. Prioritize as emergent.

🚑 Triage Record (Structured JSON Data):
{
    "patient_id": "12345",
    "age": 67,
    "gender": "male",
    "s

# **Conclusion**



To learn more, visit [Fireworks AI Docs](https://docs.fireworks.ai) or join our [Discord community](https://discord.gg/fireworks-ai).